In [1]:
import pandas as pd
import numpy as np

In [2]:
file_path = 'Local Products Exportation Morocco.csv'
data = pd.read_csv(file_path, delimiter=',')

data

,Period,Agricultural products,Cotton,Leather,Processed food and agro-based products,Total
0,2000-Q1,332441.0,169.0,58295.0,304941.0,695846.0
1,2000-Q2,224801.0,244.0,55548.0,200588.0,481181.0
2,2000-Q3,127046.0,23.0,63902.0,216606.0,407577.0
3,2000-Q4,232257.0,0.0,58893.0,277342.0,568492.0
4,2001-Q1,301359.0,72.0,65345.0,240431.0,607207.0
...,...,...,...,...,...,...
89,2022-Q2,1935508.0,32.0,99684.0,1011092.0,3046316.0
90,2022-Q3,1011122.0,25.0,108998.0,851474.0,1971619.0
91,2022-Q4,1644487.0,85.0,112392.0,829201.0,2586165.0
92,2023-Q1,2037322.0,467.0,112812.0,1050748.0,3201349.0


In [3]:
def last_day_month(month):
    if month == 6 or month == 9:
        return 30
    else:
        return 31

def convert_quarter_to_date(quarter):
    year, q = quarter.split('-Q')
    month = (int(q) - 1) * 3 + 3
    day = last_day_month(month)
    return pd.Timestamp(f"{year}-{month:02d}-{day:02d}")

data['Period'] = data['Period'].apply(convert_quarter_to_date)

data.head()

,Period,Agricultural products,Cotton,Leather,Processed food and agro-based products,Total
0,2000-03-31,332441.0,169.0,58295.0,304941.0,695846.0
1,2000-06-30,224801.0,244.0,55548.0,200588.0,481181.0
2,2000-09-30,127046.0,23.0,63902.0,216606.0,407577.0
3,2000-12-31,232257.0,0.0,58893.0,277342.0,568492.0
4,2001-03-31,301359.0,72.0,65345.0,240431.0,607207.0


In [7]:
from sklearn.preprocessing import MinMaxScaler

data['Period'] = pd.to_datetime(data['Period'])

data['year'] = data['Period'].dt.year
data['month'] = data['Period'].dt.month
data['day'] = data['Period'].dt.day

closedf = data[['Period','Total']]
print("Shape of close dataframe:", closedf.shape)

closedf = closedf[closedf['Period'] > '1999-02-19']
close_stock = closedf.copy()
print("Total data for prediction: ",closedf.shape[0])

del closedf['Period']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)

Shape of close dataframe: (94, 2)
Total data for prediction:  94
(94, 1)


In [13]:
training_size=int(len(closedf)*0.60)
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]

In [19]:
time_step = 15
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

In [21]:
temp_input

[0.6449493590689359,
 0.7942806557718511,
 0.495913489494285,
 0.4386245801619469,
 0.7077941036077866,
 0.855756229225068,
 0.673966135233417,
 0.5466287112816017,
 0.8164692813066836,
 1.0,
 0.9160459283302813,
 0.5429617350702551,
 0.756303168638206,
 0.969866085764127,
 0.8191302083785355]

In [23]:
lst_output=[]
n_steps=time_step
i=0
pred_q = 30

In [26]:
import joblib

lr = joblib.load('linear_regression_model.pkl')

In [27]:
while(i<pred_q):

    if(len(temp_input)>time_step):

        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))

        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        x_input = x_input.reshape(x_input.shape[0], -1)
        yhat = lr.predict(x_input)
        #print("{} day output {}".format(i,yhat))
        temp_input.append(yhat[0])
        temp_input=temp_input[1:]
        #print(temp_input)

        lst_output.extend(yhat.tolist())
        i=i+1

    else:

        x_input = x_input.reshape((1, n_steps,1))
        x_input = x_input.reshape(x_input.shape[0], -1)
        yhat = lr.predict(x_input)
        temp_input.append(yhat[0])

        lst_output.extend(yhat.tolist())
        i=i+1

print("Output of predicted next quarterly time series: ", len(lst_output))

Output of predicted next quarterly time series:  30


In [28]:
last_q=np.arange(1,time_step+1)
q_pred=np.arange(time_step+1,time_step+pred_q+1)

In [36]:
from itertools import cycle
import plotly.express as px

temp_mat = np.empty((len(last_q)+pred_q+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_q_value = temp_mat
next_predicted_q_value = temp_mat

last_original_q_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_q_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_q_value':last_original_q_value,
    'next_predicted_q_value':next_predicted_q_value
})

names = cycle(['Valeur des 15 dernières','Valeur prévue des 30 prochaines'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_q_value'],
                                                      new_pred_plot['next_predicted_q_value']],
              labels={'value': 'valeur','index': 'Séries chronologiques trimestrielles'})
fig.update_layout(title_text='15 dernières séries chronologiques trimestrielles VS 30 prochaines séries chronologiques trimestrielles',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Valeur en USD')

fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [37]:
lstmdf=closedf.tolist()
lstmdf.extend((np.array(lst_output).reshape(-1,1)).tolist())
lstmdf=scaler.inverse_transform(lstmdf).reshape(1,-1).tolist()[0]

names = cycle(['Valeur'])

fig = px.line(lstmdf,labels={'value': 'valeur','index': 'Séries chronologiques trimestrielles'})
fig.update_layout(title_text='Tracer la valeur entière avec prédiction',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Stock')

fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()